## To make it work straight away, make sure this notebook.ipynb file is located within a similar file structure as below:

### Suggested folder structure:
<br>
<li>/Datathon2019/ </li>
<li><ul>    > image-analysis-workbooks/   (git repo - this .ipynb file should be in here)</li>
<br>    > phase-01                    (data folder downloaded and unzipped from medium.com  </ul>

In [ ]:
from PIL import Image, ImageDraw
from glob import glob
import os
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

## OVERRIDE SETTINGS FOR YOUR ENV

In [ ]:
DATA_DIR= '../data/phase-01'

### from the demo.py file, showing how to use PIL Image to open a mask file

In [ ]:
TILE_X = 7680
TILE_Y = 10240
IS_IN_MASK_PIXEL_VALUE = (0, 0, 0, 255)

In [ ]:
# Get the physical path to the PNG image containing the mask file
def get_mask_path(tile_x, tile_y, mask_type):
    path = f"{DATA_DIR}/data/sentinel-2a-tile-{tile_x}x-{tile_y}y/masks/{mask_type}-mask.png"
    return path

In [ ]:
mask_path = get_mask_path(TILE_X, TILE_Y, "sugarcane-region")

In [ ]:
mimg = Image.open(mask_path)

In [ ]:
mimg

In [ ]:
mpixels = mimg.load()

In [ ]:
## testing out a few properties of an Image() object
print(mimg.size)
print(mimg.category)
print(mimg.format)
print(mimg.height)
print(mimg.info)
print(mimg.text)
print(mimg.width)

In [ ]:
imglist = glob('../../phase-01/data/sentinel-2a-tile-7680x-10240y/timeseries/*')
imglist.sort()
imglist

In [ ]:
i0 = Image.open(imglist[0])

### Bit of data definition from documentation to look at different types of images we have

In [ ]:
types_of_image = [i.split('/')[-1].split('-')[2] for i in imglist]
types_of_image = sorted(list(set(types_of_image)))
print(len(types_of_image)," different types of images")
b8a = types_of_image.pop(12)
types_of_image.insert(8,b8a)
print(types_of_image)

In [ ]:
descriptions = ["Coastal aerosol",'Blue','Green','Red','Vegetation Red Edge','Vegetation Red Edge','Vegetation Red Edge','NIR','Vegetation Red Edge','Water vapour','SWIR - Cirrus','SWIR','SWIR','True Colour Image']
wavelengths = [0.443,0.490,0.560,0.665,0.705,0.740,0.783,0.842,0.865,0.945,1.375,1.610,2.19,None]
resolutions = [60,10,10,10,20,20,20,10,20,60,60,20,20,None]
print(len(descriptions),len(wavelengths),len(resolutions))

### datadict variable is now dict with documentation for each image type

In [ ]:
datadict = {k:v for k,v in zip(types_of_image,[{'Description':desc,'Wavelength':wvl,'Resolution':res} 
                                               for desc,wvl,res in zip(descriptions,wavelengths,resolutions)])}
pprint(datadict)

In [ ]:
exampleimages = {t:sorted(glob(f'../../phase-01/data/sentinel-2a-tile-7680x-10240y/timeseries/*{t}*'))[0] for t in types_of_image}
exampleimages

In [ ]:
exampleimages = {k:Image.open(v) for k,v in exampleimages.items()}
exampleimages

## This looks like a better way to visualise - use matplotlib.pyplot.imshow()
<br> Had to do some preprocessing to reverse the values in the mask

In [ ]:
def reverse_mask(mask_image):
    """This is for creating a 'reversed mask' for use with matplotlib.pyplot.imshow()
    Where we want the pixels that ARE in the mask to be transparent/blank, 
    and the pixels that are not in the mask to be blanked out.
    Does take a second or two to loop through the np.array, might be a more efficient way to do it
    if this needs to be done at scale but I suspect we won't reach there just for visualisation"""
    themask = np.asarray(mask_image).copy()
    
    BLANK = [0,0,0,0]
    IS_IN_MASK_PIXEL_VALUE = [0, 0, 0, 255]
#     BLOCK = [255,255,255,255]
    
    for row in range(mask_image.height):
        for column in range(mask_image.width):
            pixel = themask[row,column]

            if (pixel == IS_IN_MASK_PIXEL_VALUE).all() == True:
                themask[row,column] = BLANK
            else:
                themask[row,column] = IS_IN_MASK_PIXEL_VALUE
    return themask


In [ ]:
revmask = reverse_mask(mimg)

In [ ]:
def multi_scatter(indict, mask="", no_cols=4, clrmap='seismic', mask_alpha=0.4, infodict=datadict):

    no_rows = int(np.ceil(len(indict.keys())/no_cols))
    plotlen = int(round((15/no_cols)*no_rows))
    fig, axes = plt.subplots(nrows=no_rows, ncols=no_cols, figsize = (15,plotlen), sharey=True)

    axes = axes.flatten()

    for i, name, image in zip(range(len(indict)),indict.keys(),indict.values()):

        axes[i].imshow(np.asarray(image), cmap=clrmap)
        if len(mask) > 0:
            axes[i].imshow(mask, alpha=mask_alpha)
        axes[i].set(title=f"{name}: {infodict[name]['Description']}")

    fig.subplots_adjust(wspace= 0.1, hspace=0.2)

#     plt.savefig("xxxxxxxxx.png")
    plt.show()

In [ ]:
multi_scatter(exampleimages, mask=revmask, no_cols=3, mask_alpha=0.7)
## clrmap as 'seismic' means red = high, blue = low, white should be centred on zero. 
## see https://matplotlib.org/users/colormaps.html

In [ ]:
exampleimages['TCI']

## Run each of the cells below in succesion to get info and example image for each of the 14 types of image
<br> admittedley it's not super exciting, mostly black and white masks except the TCI one at the end

In [ ]:
# t = 0
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

In [ ]:
# t += 1
# typex = types_of_image[t]
# print(typex,datadict[typex])
# exampleimages[typex]

## Warning - running below cell will open up 14 image files in whatever the default image viewing tool your computer runs (opened up 14 separate Preview windows on my Mac)
<br> and none of them are very interesting, it seems to reverse the black in the non-TCI image to white and show basically nothing

In [ ]:
# for t in range(len(exampleimages)):
#     typex = types_of_image[t]
#     print(typex,datadict[typex])
#     exampleimages[typex].show()